In [2]:
from pint import UnitRegistry
import pandas as pd


# helpef functions, constants, etc 

In [68]:
class ComponentReportClass:
    def __init__(self, depth, length, width):
        self.depth = depth
        self.length = length
        self.width = width 
        self.rebar = None
        self.rebar_temp_length = None
    # dimensional calcs   
    def calc_volume(self):
        return (self.depth).to("yards") * (self.width).to("yards") * (self.length).to("yards")
    def calc_top_face_area(self):
        return self.length * self.width 
    def calc_sides_face_area(self):
        return 2 * (self.depth * self.width ) + 2 * (self.depth * self.length )
    # conversion calcs
    def to_cubic_yards(a):
        return a/27

    # basic work calcs
    def calc_subgrade_prep(self):
        bottom = self.calc_top_face_area()
        sides = self.calc_sides_face_area()
        return bottom + sides
    def calc_formwork(self):
        return self.calc_sides_face_area()
    def calc_excavation(self):
        # TODO add benching...
        return self.calc_volume()
    def calc_concrete(self):
        return self.calc_volume()* 1.08


    # rebar calcs
    def calc_rebar(self, quant, length):
        """quant and length need to be arrays """
        quant = to_list(quant)
        length = to_list(length)

        h = [a*b for a,b in zip(quant, length)]
        self.rebar = sum(h)*1.20
        return self.rebar

    def calc_rebar_template(self, pilecap=False):
        """quant and length need to be arrays """
        rebar_temp_length = 0
        if pilecap==True:
            rebar_temp_length = self.length * 2 + self.width * 2
        else:
            rebar_temp_length = self.length

        self.rebar_temp_length = rebar_temp_length
        return self.rebar_temp_length

    # reports
    def report_basic(self):
        report = {
            "subgrade": self.calc_subgrade_prep(),
            "excavation": self.calc_excavation(),
            "formwork": self.calc_formwork(),
            "rebar_template": self.rebar_temp_length if self.rebar_temp_length else 0*ft,
            "concrete": self.calc_concrete(),
            "rebar": self.rebar if self.rebar else 0*ft
        }
        return report 


    

In [176]:
u = UnitRegistry()
ft = u.feet
inc = u.inches
lbs = u.pounds

def to_list(a):
    a = [a] if type(a) != list else a
    return a

def calc_component_totals(components, fx):
    """components are defined components
    fx is a function that takes in a list of components """
    c_reports = [c.report_basic() for c in components]

    sum_components = {}
    for k in c_reports[0].keys():
        sum_components[k] = fx(c_reports, k)

    return sum_components

def to_df(a_dict):
    t1 = { f"{k} [{str(v.units)}]": round(v.magnitude,2) for k,v in a_dict.items()}
    return pd.DataFrame(t1, index=[0])

In [177]:
(8) * (u.pounds/ u.foot)

8 <Unit('pound / foot')>

rebar types

In [178]:
rebar_dict = {
    "4" : 0.668 * (lbs/ft),
    "5" : 1.043 * (lbs/ft),
    "6": 1.502* (lbs/ft),
    "9": 3.4 * (lbs/ft)
}

In [179]:
def calc_wall_rebar(spacing, rtype, l1, l2):
    """ input integers in inches  """
    spacing = (spacing*inc).to("feet")
    a = (l1 / spacing) * l2
    b = (l2 / spacing) * l1
    weight = rebar_dict[str(rtype)] * (a + b)
    return weight

# Start Calcs

pile caps

In [180]:
# pile caps 
pc1 = ComponentReportClass(2*ft, 4*ft, 4*ft) # reported in ft!

pc2 = ComponentReportClass(2*ft + 9*inc, 
                            17*ft + 2*inc, 
                            10*ft + 6*inc) 
# pc1.calc_rebar([16, 16], [rebar_dict["5"], rebar_dict["5"]]) 
# pc2.calc_rebar([14+16, 16], [rebar_dict["5"], rebar_dict["5"]])
pc1.calc_rebar_template(pilecap=True)
pc2.calc_rebar_template(pilecap=True)




55.333333333333336 <Unit('foot')>

In [181]:
# pc_1 rebar 

# A and C -> (8) #9
# B and D -> (8) #9
spacing = 12*ft
pc1_length = 4*ft
pc1_rebar = {
    "ac" : 8 * rebar_dict["9"] * 4*ft,
    "bd" : 8 * rebar_dict["9"] * 4*ft,
    "s_up": pc1_length/spacing * rebar_dict["5"] * 2*ft,
    "s_mid": pc1_length/spacing * rebar_dict["5"] * 8*ft,
    "s_bot": pc1_length/spacing * rebar_dict["5"] * 6*ft,
}

pc6_length = 17*ft + 2*inc
pc6_width = 10*ft + 6*inc
pc6_depth = 2*ft + 9*inc

pc6_rebar = {
    "ac" : 8 * rebar_dict["9"] * pc6_length,
    "bd" : 8 * rebar_dict["9"] * pc6_width,
    "s_up": (pc6_length/spacing) * rebar_dict["5"] * 2*ft,
    "s_mid": (pc6_length/spacing) * rebar_dict["5"] * (pc6_length * 1.7),
    "s_bot": (pc6_length/spacing) * rebar_dict["5"] * (pc6_length * 1.4),
}



pc1.rebar = sum(pc1_rebar.values())*1.2
pc2.rebar = sum(pc6_rebar.values())*1.2
pc = calc_component_totals([pc1, pc2], lambda list, key: 2*list[0][key] + list[1][key] )
to_df(pc)


,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [pound]
0,428.42,20.73,216.17,87.33,22.39,1537.49


grade beams 

In [183]:
# gradebeams
gb_length = 6 * (2*ft + (6*inc + (1/8)*inc))
gb_width = gb_height = (12*inc).to("feet")
gb1 = ComponentReportClass(gb_width, gb_length,  gb_height) 


In [184]:

gb_rebar = {
    "bottom" : 4*2 * rebar_dict["5"] * gb_length,
    "left_and_right" : 4*4 * rebar_dict["5"] * gb_width,
    "interior": 40*(gb_length/spacing) * rebar_dict["4"] * gb_length,
}

gb1.rebar = sum(gb_rebar.values())*1.2

In [185]:

gb1.calc_rebar_template(pilecap=False)
gb = calc_component_totals([gb1], lambda list, key: 2*list[0][key])
to_df(gb)

,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [pound]
0,94.38,1.12,64.25,30.12,1.21,1554.13


# calc all component totals

In [ ]:
# all 
def fx(list, key):
    # 2* pc1 + 1*pc2 * 2*gb1
   return 2*list[0][key] + list[1][key] + 2*list[2][key]

t = calc_component_totals([pc1, pc2, gb1], fx) # TODO update excel!!!
to_df(t)

,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [foot]
0,522.79,21.84,280.42,0,23.59,492.5


# Elevator pit 

- subgrade prep => outer surfaces of walls 
- excavation => enclosed volume + steping, (18" minimum depth, 1:2 ratio)
- formwork =>  outer + inner surfaces of walls 
- rebar template => ???
- concrete => volume of walls, minus overlap
- rebar => TODO 


In [ ]:
elev_walls = {
    "depth" : 5*ft,
    "length": 9*ft + 10*inc + (1/4)*inc,
    "width": 7*ft + 2*inc + (7/8)*inc,
    "side_thickness": 1*ft,
    "floor_thickness": (10*inc).to("feet"),
}

sump_walls = {
    "depth" : 2*ft,
    "length": 4*ft,
    "width": 4*ft,
    "side_thickness": (8*inc).to("feet"),
    "floor_thickness": 3*ft,
}

# component dictionary to fill 
ep = {
    "subgrade": 0,
    "excavation": 0,
    "formwork": 0,
    "rebar_template": 0,
    "concrete": 0,
    "rebar": 0
}

In [ ]:
def calc_wall_areas(d):
    wall_area = {
    "wide_sides": d["length"] * d["depth"],
    "short_sides": d["width"] * d["depth"],
    "floor": d["length"] * d["width"]
    }
    return wall_area

In [ ]:
elev_wall_area = calc_wall_areas(elev_walls)
sump_wall_area = calc_wall_areas(sump_walls)

print(elev_wall_area, "\n", sump_wall_area)

{'wide_sides': <Quantity(49.2708333, 'foot ** 2')>, 'short_sides': <Quantity(36.1979167, 'foot ** 2')>, 'floor': <Quantity(71.3400608, 'foot ** 2')>} 
 {'wide_sides': <Quantity(8, 'foot ** 2')>, 'short_sides': <Quantity(8, 'foot ** 2')>, 'floor': <Quantity(16, 'foot ** 2')>}


subgrade and formwork 

In [ ]:
side_walls_area = 2*elev_wall_area["wide_sides"] + 2*elev_wall_area["short_sides"] + 4*sump_wall_area["wide_sides"] 

In [ ]:
# subgrade prep 
# outer wall area = side wall areas for elev and sump + sump floor area + (elev floor - sump floor) area 

subgrade_prep_area = side_walls_area + sump_wall_area["floor"] + ( elev_wall_area["floor"] - sump_wall_area["floor"])
ep["subgrade"] =  subgrade_prep_area

In [ ]:
# formwork 
# formwork area = 2* side wall values... and no bottom values, technically inner formwork is smaller than outer 
formwork_area = 2*side_walls_area
ep["formwork"] = formwork_area

concrete 

In [ ]:
# concrete -> calc wall volume and sum, then subtract volume of sump pit floor because it is taken out of elevator pit floor 

def calc_wall_volumes(d):
    return {
    "wide_sides": (d["length"] * d["depth"] * d["side_thickness"]).to("yard ** 3") ,
    "short_sides": (d["width"] * d["depth"] * d["side_thickness"]).to("yard ** 3"),
    "floor": (d["length"] * d["width"] * d["floor_thickness"]).to("yard ** 3")
    }

In [ ]:
elev_volumes  = calc_wall_volumes(elev_walls)
sump_volumes = calc_wall_volumes(sump_walls)

print(elev_volumes, "\n", sump_volumes)

{'wide_sides': <Quantity(1.82484568, 'yard ** 3')>, 'short_sides': <Quantity(1.34066358, 'yard ** 3')>, 'floor': <Quantity(2.20185373, 'yard ** 3')>} 
 {'wide_sides': <Quantity(0.197530864, 'yard ** 3')>, 'short_sides': <Quantity(0.197530864, 'yard ** 3')>, 'floor': <Quantity(1.77777778, 'yard ** 3')>}


In [ ]:
side_volumes = 2*elev_volumes["wide_sides"] + 2*elev_volumes["short_sides"] + 4* sump_volumes["wide_sides"]

floor_volumes = (elev_volumes["floor"] - sump_volumes["floor"]) + sump_volumes["floor"]

concrete_volume =  side_volumes + floor_volumes

ep["concrete"] = concrete_volume*1.08 # for errors 
ep["concrete"] 

10.068835358796298 <Unit('yard ** 3')>

excavation 

In [ ]:
import math

In [ ]:
# required benching, every 18" 
total_depth = sump_walls["depth"] + sump_walls["floor_thickness"] + elev_walls["depth"] + elev_walls["floor_thickness"]
bench_depth = (18*inc).to("feet")

In [ ]:
bench_area = bench_depth * (2*bench_depth)
steps = math.ceil(total_depth / bench_depth)
side_bench_area = sum(range(steps+1)) * bench_area
side_bench_area

162.0 <Unit('foot ** 2')>

In [ ]:
bench_volume = 2 * side_bench_area * elev_walls["length"] + 2 * side_bench_area * elev_walls["width"] 
bench_volume = bench_volume.to("yards ** 3")

In [ ]:
inner_volume_elev = elev_walls["depth"] * elev_walls["width"] * elev_walls["length"]
inner_volume_sump = sump_walls["depth"] * sump_walls["width"] * sump_walls["length"]
inner_volume = (inner_volume_elev + inner_volume_sump).to("yards ** 3")

In [ ]:
excav_volume = bench_volume + inner_volume 
ep["excavation"] = excav_volume

rebar

In [ ]:
# lbs/ft

rebar_dict = {
    "4" : 0.668 * (lbs/ft),
    "5" : 1.043 * (lbs/ft),
    "6": 1.502* (lbs/ft)
}
rebar_dict

{'4': 0.668 <Unit('pound / foot')>,
 '5': 1.043 <Unit('pound / foot')>,
 '6': 1.502 <Unit('pound / foot')>}

In [ ]:
def calc_wall_rebar(spacing, rtype, l1, l2):
    """ input integers in inches  """
    spacing = (spacing*inc).to("feet")
    a = (l1 / spacing) * l2
    b = (l2 / spacing) * l1
    weight = rebar_dict[str(rtype)] * (a + b)
    return weight


In [ ]:
# elevator 
elev_rebar = {
"floor": 2*calc_wall_rebar(12, 4, elev_walls["length"] , elev_walls["width"] ),
"wide": 2*calc_wall_rebar(12, 5, elev_walls["length"] , elev_walls["depth"] ),
# two types of rebar in the short walls 
"short": calc_wall_rebar(12, 5, elev_walls["width"] , elev_walls["depth"] ) + calc_wall_rebar(12, 6, elev_walls["width"] , elev_walls["depth"] ),
}
# multiply for num walls 
elev_rebar["wide"]*=2
elev_rebar["short"]*=2
elev_rebar

{'floor': 190.62064236111115 <Unit('pound')>,
 'wide': 411.1158333333334 <Unit('pound')>,
 'short': 368.4947916666667 <Unit('pound')>}

In [ ]:
# ignore floor of sump pit (+/- assumption)
sump_rebar = {
"wide": 2*calc_wall_rebar(12, 5, sump_walls["length"] , elev_walls["depth"] ),
# one wide wall has only one rebar 
"wide_2": calc_wall_rebar(9, 4, sump_walls["length"] , elev_walls["depth"] ),
# two types of rebar in the short walls 
"short": calc_wall_rebar(12, 5, sump_walls["width"] , sump_walls["depth"] ) + calc_wall_rebar(12, 6, elev_walls["width"] , sump_walls["depth"] ), 
}
sump_rebar["short"]*=2
sump_rebar

{'wide': 83.44 <Unit('pound')>,
 'wide_2': 35.626666666666665 <Unit('pound')>,
 'short': 120.36683333333335 <Unit('pound')>}

In [ ]:
sum_rebar = sum(elev_rebar.values()) + sum(sump_rebar.values())
ep["rebar"] = sum_rebar*1.20

### finish

In [ ]:
ep

{'subgrade': 274.2775607638889 <Unit('foot ** 2')>,
 'excavation': 219.52130754886832 <Unit('yard ** 3')>,
 'formwork': 405.87500000000006 <Unit('foot ** 2')>,
 'rebar_template': 0,
 'concrete': 10.068835358796298 <Unit('yard ** 3')>,
 'rebar': 1451.5977208333336 <Unit('pound')>}